In [66]:
from queue import PriorityQueue
from copy import deepcopy

SIDE = 3
puzzleSize = SIDE * SIDE   # 8-puzzle is 3 x 3

'''
The search node class
- state is a list representing the puzzle arrangement in row-major order. 
   0 represents blank. ex> [1, 2, 3, 8, 0, 4, 7, 6, 5]
- parent is a parent node
- operator is one of U, D, L, R
- evaluation value of node f = g + h
- moves record the path (sequence of actions)
'''
class Node:
    def __init__(self, state, parent, operator, g, h):
        self.state = state
        self.parent = parent
        self.operator = operator
        self.g = g
        self.h = h
        self.f = self.g + self.h
        if parent is None:
            self.moves = operator
        else:
            self.moves = parent.moves + '-' + operator
    def __lt__(self, other):
        return self.f < other.f

    '''
ASTAR performs A* search
- OPEN is a PriorityQueue of (node.f, node).
- CLOSED is a list of states
- When a next state is in OPEN, it should be updated.
   But instead, here we just put it into OPEN,
   and when it is chosen, we ignore the state if it is in CLOSED
xpand returns list of next (child) nodes
'''
def ASTAR(startNode):
    
    # initial OPEN and CLOSED
    OPEN = PriorityQueue()
    OPEN.put((startNode.f, startNode))
    CLOSED = []
    
    while not(OPEN.empty()):
        ##################################################################
        
        # Choose the best node from OPEN
        # 우선 순위 큐에서 우선순위가 높은 노드를 get
        curr = OPEN.get()[1]
        # If it is in CLOSED, ignore
        # CLOSED에 있지 않다면
        if curr.state not in CLOSED:
            # Test if it is a goal
            # goal에 도달한 노드이면 반환
            if goaltest(curr.state):
                print("CLOSED: ", len(CLOSED))
                return curr
            # Expand the node
            # 현재노드를 방문하였음을 표시, CLOSED에 넣음
            # 노드를 확장하고 CLOSED에 들어 있는 것을 제외하고
            # OPEN 큐에 넣어준 뒤 반복
            else:
                CLOSED.append(curr.state)
                node = expand(curr)
                for i in node:
                    if i.state not in CLOSED:
                        OPEN.put((i.f, i))
        ##################################################################
                
    return None 
                
'''
goaltest returns 1 if state is same as the goalState
'''
def goaltest(state):
    if state == goalState:
        return 1
    else:
        return 0

'''
heuristic returns estimated distant from state to the goalState
'''

# def heuristic(state):
    
#     ##################################################################
#     h = 0
#     # compute h 
#     # 빈 칸을 의미하는 0인 칸을 제외하고 goal state와 일치하지 않는 칸을 카운트
#     for i in range(9):
#         if state[i] == 0:
#             continue
#         if goalState[i] != state[i]:
#             h = h + 1
    
#     return h
#     ##################################################################

def heuristic(state):
    h = 0
    for i in range(9):
        if state[i] == 0:
            continue
        if goalState[i] != state[i]:
            for j in range(9):
                if goalState[j] == state[i]:
                    h += abs((j // 3 - i // 3) + abs(j % 3 - i % 3))
    
    return h
'''
expand returns list of next (child) nodes
'''
def expand(node):
    nodeList = []
    state = node.state
    blank = state.index(0)   # location of blank
    row = int(blank / SIDE)
    col = blank % SIDE
    
    # up
    if not (row == 0):
        childState = deepcopy(state)
        childState[blank] = childState[blank-SIDE]
        childState[blank-SIDE] = 0
        nodeList.append(Node(childState, node, "U", node.g+1, heuristic(childState)))
    # down
    if not (row == SIDE-1):
        childState = deepcopy(state)
        childState[blank] = childState[blank+SIDE]
        childState[blank+SIDE] = 0
        nodeList.append(Node(childState, node, "D", node.g+1, heuristic(childState)))
    # left
    if not (col == 0):
        childState = deepcopy(state)
        childState[blank] = childState[blank-1]
        childState[blank-1] = 0
        nodeList.append(Node(childState, node, "L", node.g+1, heuristic(childState)))
    # right
    if not (col == SIDE-1):
        childState = deepcopy(state)
        childState[blank] = childState[blank+1]
        childState[blank+1] = 0
        nodeList.append(Node(childState, node, "R", node.g+1, heuristic(childState)))

    return nodeList

'''
printState print 3x3 puzzle state
'''
def printState(state):
    for i in range(3):
        for j in range(3):
            print(state[i*3 + j], end="\t")
        print("")
    print("")
    
'''
printSolutionPath show solution path recursively
'''
def printSolutionPath(node):
    if (node == None): return
    printSolutionPath(node.parent)
    printState(node.state)


In [67]:
'''
The start and goal states
'''
startState = [1, 2, 3, 
              8, 0, 4, 
              7, 6, 5]
goalState = [7, 8, 1,
             6, 0, 2,
             5, 4, 3]

print("Start state")
printState(startState)
print("Goal state")
printState(goalState)

'''
Make start node and do A* search
'''
startNode = Node(startState, None, "Start", 0, heuristic(startState))
print("Start A* search")
goalNode = ASTAR(startNode)
print(" Finished")

'''
If succeeded, show the solution path
'''
if (goalNode == None):
    print("A* Failed ")
else:
    print("A* Succeeded ")
    print("solution path: ", goalNode.moves)
    printSolutionPath(goalNode)


Start state
1	2	3	
8	0	4	
7	6	5	

Goal state
7	8	1	
6	0	2	
5	4	3	

Start A* search
CLOSED:  84
 Finished
A* Succeeded 
solution path:  Start-R-U-L-L-D-D-R-R-U-U-L-L-D-D-R-U
1	2	3	
8	0	4	
7	6	5	

1	2	3	
8	4	0	
7	6	5	

1	2	0	
8	4	3	
7	6	5	

1	0	2	
8	4	3	
7	6	5	

0	1	2	
8	4	3	
7	6	5	

8	1	2	
0	4	3	
7	6	5	

8	1	2	
7	4	3	
0	6	5	

8	1	2	
7	4	3	
6	0	5	

8	1	2	
7	4	3	
6	5	0	

8	1	2	
7	4	0	
6	5	3	

8	1	0	
7	4	2	
6	5	3	

8	0	1	
7	4	2	
6	5	3	

0	8	1	
7	4	2	
6	5	3	

7	8	1	
0	4	2	
6	5	3	

7	8	1	
6	4	2	
0	5	3	

7	8	1	
6	4	2	
5	0	3	

7	8	1	
6	0	2	
5	4	3	

